<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-2/blob/main/CS6910_Assignment2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS6910-Assignment2-Part B
Using Pretrained model for Image Classification

In [10]:
!pip install -q wandb
import wandb
from wandb.keras import WandbCallback

import numpy as np
import pandas as pd
import math
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [11]:
def get_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"

    # Training dataset:
    train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='training'
    )
    # Validation dataset:
    valid_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='validation'
    )
    
    #return train_data, valid_data
    return train_dataset, valid_dataset


In [12]:
def get_augmented_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"
    training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        horizontal_flip=True,
                                        zoom_range=0.2,
                                        fill_mode="nearest",
                                        validation_split = 0.1)

    # Validation data is not being augmented
    validation_data_augmentation=ImageDataGenerator(
        validation_split=0.1
    )

    train_aug=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
    valid_aug=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')

    return train_aug, valid_aug


#Transfer Learning 
1. Reference 1: https://www.tensorflow.org/tutorials/images/transfer_learning
2. Reference 2: https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751

In [13]:
def inceptionv3(image_shape):
  model=tf.keras.applications.InceptionV3(input_shape=image_shape,
                                          include_top=False,
                                          weights='imagenet')
  return model

def resnet50(image_shape):
  model=tf.keras.applications.ResNet50(input_shape=image_shape,
                                      include_top=False,
                                      weights='imagenet')
  return model

def inceptionresnetv2(image_shape):
  model=tf.keras.applications.InceptionResNetV2(input_shape=image_shape,
                                                include_top=False,
                                                weights='imagenet')
  return model

def xception(image_shape):
  model=tf.keras.applications.Xception(input_shape=image_shape,
                                      include_top=False,
                                      weights='imagenet')
  return model

def transferlearning_model(name,dense_units,dropout,freeze_percent,batch_norm):
  
  image_shape=(256,256,3)
  # calculating the number of layers in each model, and then freezing those layers
  # freezing ensures that those layers are not trained, to maintain the generality

  if name  == "InceptionV3":
    model=inceptionv3(image_shape)
    num_layers=len(model.layers)
    k=math.ceil(freeze_percent*num_layers)
    
  elif name == "InceptionResNetV2":
    model=inceptionresnetv2(image_shape)
    num_layers=len(model.layers)
    k=math.ceil(freeze_percent*num_layers)
  
  elif name == "ResNet50":
    model=resnet50(image_shape)
    num_layers=len(model.layers)
    k=math.ceil(freeze_percent*num_layers)

  elif name == "Xception":
    model=xception(image_shape)
    num_layers=len(model.layers)
    k=math.ceil(freeze_percent*num_layers)
  
  else:
    print("Enter a valid model name")

  # Freezing the k layers, making them un trainable
  i=0
  for layer in model.layers:
    layer.trainable=False
    i=i+1
    if(i==k):
      break

  # Adding a dense layer on top of the convolution layers.
  a=layers.Flatten()(model.output)
  a=layers.Dense(dense_units,layers.ReLU())(a)
  if batch_norm:
    a=layers.BatchNormalization()(a)
  a=layers.Dropout(dropout)(a)
  a=layers.Dense(10,tf.nn.softmax)(a)

  final_model=Model(model.input,a)
  
  return final_model

In [14]:
model=transferlearning_model("InceptionV3",32,0.2,0.6,True)

# Wandb Sweeping

In [15]:
hyperparameters={
    'model_name':{'values': ["InceptionV3","InceptionResNetV2","ResNet50","Xception"]},
    'data_augmentation':{'values':[False,True]},
    'batch_normalization':{'values':[False,True]},
    'freeze_percent':{'values':[0.6,0.7,0.8,0.9,1.0]},
    'dense_units':{'values':[32,64,128]},
    'dropout_final_layer':{'values':[0.2,0.3,0.4,0.5]},
    'learning_rate':{'values':[0.001,0.00001,0.0005]}
}
sweep_config = {
      'method' : 'bayes',
      'metric' :{
          'name': 'val_acc',
          'goal': 'maximize'
      },
      'parameters': hyperparameters
    }

In [16]:
def wandb_train(config=sweep_config):
  # initializing wandb with the above configuration
  wandb.init(config=config)
  
  # collecting the configuration information
  config=wandb.init().config
  
  # setting up the name for each run example : Model_Xception_tune_False_aug_True
  run_name='Model_{}_freeze_percent_{}_Data_aug_{}_lr_{}_dropout_{}_dense_units_{}_batch_norm_{}'.format(config.model_name,
                                                                             config.freeze_percent,
                                                                             config.data_augmentation,
                                                                             config.learning_rate,
                                                                             config.dropout_final_layer,
                                                                             config.dense_units,
                                                                             config.batch_normalization                                                                            
                                                                             )
  wandb.run.name=run_name

  # caling the model 
  model=transferlearning_model(config.model_name,
                               config.dense_units,
                               config.dropout_final_layer,
                               config.freeze_percent,
                               config.batch_normalization)

  learning_rate=config.learning_rate
  
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  
  # Getting data 
  train_data,val_data=get_data()
  train_aug_data,val_aug_data=get_augmented_data()

  if config.data_augmentation == False:
    history = model.fit(train_data, epochs=10, validation_data=val_data,
                        callbacks = [wandb.keras.WandbCallback()])
  else:
    history = model.fit(train_aug_data, steps_per_epoch=62, epochs=10,
                        validation_data=val_aug_data, validation_steps=62,
                        workers=10, callbacks = [wandb.keras.WandbCallback()])  
  
  wandb.run.finish()

In [17]:
!wandb login --relogin
entity_name = "safi-vamsi-cs6910"
project_name = "Assignment 2"

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_id=wandb.sweep(sweep_config,entity=entity_name,project=project_name)
wandb.agent(sweep_id,wandb_train)